Import the necessary libraries

The code below is a series of import statements in Python, which are used to include various libraries and modules necessary for the script to function. Each import statement brings in specific functionality that will be used later in the code.

1. `import numpy as np`: This line imports the NumPy library and assigns it the alias `np`. NumPy is a fundamental package for scientific computing in Python, providing support for arrays, matrices, and a wide range of mathematical functions to operate on these data structures.

2. `import pandas as pd`: This line imports the Pandas library and assigns it the alias `pd`. Pandas is a powerful data manipulation and analysis library that provides data structures like DataFrames, which are essential for handling and analyzing structured data.

3. `from fancyimpute import BiScaler`: This line imports the `BiScaler` class from the `fancyimpute` library. `fancyimpute` is a library for matrix completion and imputation algorithms. The `BiScaler` class is used for scaling rows and columns of a matrix to have zero mean and unit variance.

4. `from soft_impute import SoftImpute`: This line imports the `SoftImpute` class from the `soft_impute` module. `SoftImpute` is an algorithm for matrix completion that uses soft-thresholding of singular values to handle missing data in matrices.

5. `from functionsCF import GenerateTrainingSet`: This line imports the `GenerateTrainingSet` function from a custom module named `functionsCF`. This function is likely used to generate a training dataset, although the specific details would depend on the implementation within the `functionsCF` module.

The `# type: ignore` comments are used to instruct type checkers to ignore potential type errors in these import statements. This can be useful when working with libraries or modules that may not have complete type annotations or when integrating with custom code.

In [1]:
import numpy         as np                  # type: ignore
import pandas        as pd                  # type: ignore
from fancyimpute import BiScaler            # type: ignore
from soft_impute import SoftImpute          # type: ignore
from functionsCF import GenerateTrainingSet # type: ignore

The code below reads a dataset from a CSV file named `MusicRatings.csv` and stores the data in a variable named `rating`. This dataset contains music ratings and includes 289,409 rows with columns: `userId`, `songId`, `rating`, and `timestamp`.

The code uses the `read_csv` function from the Pandas library to read the CSV file. The `pd.read_csv('MusicRatings.csv')` function call reads the entire CSV file into a Pandas DataFrame. The `.values` attribute is then used to extract the underlying NumPy array from the DataFrame, which contains only the values without any index or column labels.

By storing the data in a NumPy array, the code can efficiently perform numerical operations and manipulations on the dataset. This approach is useful when the focus is on the raw data values rather than the DataFrame's additional features like indexing and column names. The variable `rating` now holds the entire dataset in a format that is ready for further analysis or processing.

In [3]:
rating = pd.read_csv('files/MusicRatings.csv').values

The code below configure the printing options for NumPy arrays and then print a subset of the `rating` array.

1. `np.set_printoptions(precision=10, suppress=True)`: This line sets the printing options for NumPy arrays. The `precision` parameter is set to 10, which means that floating-point numbers will be displayed with up to 10 digits after the decimal point. The `suppress` parameter is set to `True`, which ensures that small floating-point numbers are printed in fixed-point notation rather than scientific notation. This makes the output more readable by avoiding the use of exponential notation for small values.

2. `print(rating[:5, :])`: This line prints the first five rows of the `rating` array. The slicing operation `rating[:5, :]` selects all columns (`:`) of the first five rows (`:5`). The `print` function then outputs these rows to the console. Given the previous `set_printoptions` call, the floating-point numbers in the printed array will adhere to the specified precision and formatting rules.

Together, these lines of code ensure that the printed output of the `rating` array is both precise and easy to read, which is particularly useful for inspecting the data and verifying its contents during development and debugging.

In [4]:
np.set_printoptions(precision=10, suppress=True)
print(rating[:5, :])

[[ 526.            80.             1.477121255]
 [1403.            54.             2.204119983]
 [ 556.            80.             1.301029996]
 [1036.            54.             1.477121255]
 [2352.            80.             1.301029996]]


The code below creates a matrix filled with zeros, named `matrix_incomplete`. This matrix is intended to represent an incomplete data matrix for collaborative filtering tasks, such as those used in recommendation systems.

1. `np.zeros((...))`: This function from the NumPy library creates a new array of given shape and type, filled with zeros. The shape of the array is specified by a tuple, which in this case is determined by the lengths of unique user IDs and song IDs.

2. `len(np.unique(rating[:,0]))`: This part of the code calculates the number of unique user IDs in the `rating` array. The `rating[:,0]` expression selects the first column of the `rating` array, which contains user IDs. The `np.unique` function then finds all unique user IDs, and `len` calculates the total number of these unique IDs.

3. `len(np.unique(rating[:,1]))`: Similarly, this part calculates the number of unique song IDs in the `rating` array. The `rating[:,1]` expression selects the second column of the `rating` array, which contains song IDs. The `np.unique` function finds all unique song IDs, and `len` calculates the total number of these unique IDs.

By combining these two lengths, the code creates a matrix with dimensions corresponding to the number of unique users and unique songs. Each element in this matrix initially contains a zero, representing the absence of a rating. This matrix will likely be used to store and manipulate the ratings data, where non-zero values will represent actual ratings given by users to songs. This setup is a common preprocessing step in collaborative filtering algorithms, where the goal is to predict missing ratings based on the observed data.

In [5]:
matrix_incomplete = np.zeros((len(np.unique(rating[:,0])), len(np.unique(rating[:,1]))))

The code processes a dataset of music ratings to ensure that only music with ratings is considered and that the music IDs are adjusted accordingly.

1. **Filtering Music with Ratings**:
   Since some music tracks may not have any ratings, the code will only use the music tracks that have received ratings. This is important for ensuring that the subsequent analysis or modeling only includes relevant data.

2. **Creating an Array of Unique Music IDs**:
   ```python
   usedID = np.unique(rating[:, 1])
   ```
   This line creates an array called `usedID` that contains all unique music IDs from the `rating` array. The `rating[:, 1]` expression selects the second column of the `rating` array, which contains the music IDs. The `np.unique` function then finds and returns the unique music IDs.

3. **Replacing Music IDs with Their Positions**:
   ```python
   for i in range(len(rating[:,1])):
       rating[:,1][i] = np.where(usedID==rating[:,1][i])[0][0] + 1
   ```
   This loop iterates over each music ID in the `rating` array. For each music ID, it finds the position of that ID in the `usedID` array using the `np.where` function. The `np.where(usedID==rating[:,1][i])[0][0]` expression returns the index of the current music ID in the `usedID` array. Adding 1 to this index ensures that the IDs start from 1 instead of 0. The original music ID in the `rating` array is then replaced with this new position-based ID.

By performing these steps, the code ensures that the music IDs in the `rating` array are contiguous and correspond to the positions in the `usedID` array. This preprocessing step is crucial for algorithms that require a clean and consistent set of IDs, such as collaborative filtering methods used in recommendation systems.

In [6]:
usedID = np.unique(rating[:, 1])
for i in range(len(rating[:,1])):
    rating[:,1][i] = np.where(usedID==rating[:,1][i])[0][0] + 1

The code below initializes and populates an incomplete matrix with music ratings data. This matrix will be used for collaborative filtering tasks, such as predicting missing ratings.

1. **Initializing the Matrix with NaN Values**:
   ```python
   matrix_incomplete[:] = np.nan
   ```
   This line sets all elements of the `matrix_incomplete` array to `NaN` (Not a Number). Using `NaN` is a common practice to represent missing or undefined values in numerical computations. This step ensures that the matrix starts with no ratings, and only the available ratings will be filled in the subsequent steps.

2. **Calculating Indices for Matrix Population**:
   ```python
   indices = np.array(rating[:,0] - 1).astype(int), np.array(rating[:,1] - 1).astype(int)
   ```
   This line calculates the indices where the ratings will be placed in the `matrix_incomplete` array. The `rating[:,0] - 1` expression selects the user IDs from the `rating` array and subtracts 1 to convert them to zero-based indices (since Python uses zero-based indexing). The `astype(int)` method ensures that these indices are of integer type. Similarly, `rating[:,1] - 1` selects the music IDs and converts them to zero-based integer indices. The `np.array` function is used to create NumPy arrays for these indices. The resulting `indices` tuple contains two arrays: one for user indices and one for music indices.

3. **Populating the Matrix with Ratings**:
   ```python
   matrix_incomplete[indices] = rating[:,2]
   ```
   This line assigns the actual ratings to the corresponding positions in the `matrix_incomplete` array. The `rating[:,2]` expression selects the ratings from the `rating` array. The `indices` tuple is used to place these ratings at the correct positions in the `matrix_incomplete` array, where the first array in `indices` specifies the row indices (users) and the second array specifies the column indices (music).

By performing these steps, the code constructs a matrix where each element represents a rating given by a user to a music track. The matrix is initially filled with `NaN` values to indicate missing ratings, and the available ratings are populated based on the data in the `rating` array. This matrix is essential for collaborative filtering algorithms, which aim to predict the missing ratings based on the observed data.

In [7]:
matrix_incomplete[:] = np.nan
indices = np.array(rating[:,0] - 1).astype(int), np.array(rating[:,1] - 1).astype(int)
matrix_incomplete[indices] = rating[:,2]

The code below prepares a training matrix for a collaborative filtering task by splitting the dataset into training and validation sets and then creating a matrix to hold the training data.

1. **Generating Training and Validation Indices**:
   ```python
   train_indices, validation_indices = GenerateTrainingSet(rating[:,0], rating[:,1], 0.70)
   ```
   This line calls the `GenerateTrainingSet` function, passing the user IDs (`rating[:,0]`), music IDs (`rating[:,1]`), and a ratio of 0.70. The function splits the dataset into training and validation sets, ensuring that 70% of the data is used for training. It returns two sets of indices: `train_indices` for the training data and `validation_indices` for the validation data. These indices correspond to the positions in the original `rating` array.

2. **Copying the Incomplete Matrix**:
   ```python
   matrix_train = matrix_incomplete.copy()
   ```
   This line creates a copy of the `matrix_incomplete` array and assigns it to `matrix_train`. The `copy` method ensures that `matrix_train` is a separate object, allowing modifications without affecting the original `matrix_incomplete` array.

3. **Initializing the Training Matrix with NaN Values**:
   ```python
   matrix_train[:] = np.nan
   ```
   This line sets all elements of the `matrix_train` array to `NaN`. This step ensures that the training matrix starts with no ratings, similar to the initialization of `matrix_incomplete`.

4. **Populating the Training Matrix with Training Data**:
   ```python
   matrix_train[train_indices] = matrix_incomplete[train_indices]
   ```
   This line assigns the ratings from the `matrix_incomplete` array to the corresponding positions in the `matrix_train` array, based on the `train_indices`. The `train_indices` specify the locations of the training data, and this step populates the `matrix_train` with these ratings while leaving the rest of the matrix as `NaN`.

By performing these steps, the code prepares a training matrix (`matrix_train`) that contains only the training data, with the rest of the entries set to `NaN`. This matrix is essential for training collaborative filtering models, as it allows the model to learn from the available ratings while ignoring the missing ones. The validation indices can be used later to evaluate the model's performance on the validation set.

In [8]:
train_indices, validation_indices = GenerateTrainingSet(rating[:,0], rating[:,1], 0.70)
matrix_train = matrix_incomplete.copy()
matrix_train[:] = np.nan
matrix_train[train_indices] = matrix_incomplete[train_indices]

The code below initializes a `BiScaler` object and uses it to normalize a training matrix. This process is part of preparing the data for collaborative filtering tasks.

1. **Initializing the BiScaler Object**:
   ```python
   biscaler = BiScaler(scale_rows=False, scale_columns=False, max_iters=50, verbose=False)
   ```
   This line creates an instance of the `BiScaler` class with specific parameters:
   - `scale_rows=False`: This parameter indicates that the rows of the matrix should not be scaled.
   - `scale_columns=False`: This parameter indicates that the columns of the matrix should not be scaled.
   - `max_iters=50`: This parameter sets the maximum number of iterations for the scaling algorithm to 50.
   - `verbose=False`: This parameter disables verbose output, meaning that the algorithm will not print progress messages during execution.

   The `BiScaler` class is designed to iteratively estimate row and column centering and scaling for a matrix, which is useful for normalizing data before applying collaborative filtering algorithms.

2. **Normalizing the Training Matrix**:
   ```python
   matrix_train_normalized = biscaler.fit_transform(matrix_train)
   ```
   This line calls the `fit_transform` method of the `BiScaler` object on the `matrix_train` array. The `fit_transform` method performs two main steps:
   - `fit(X)`: This method estimates the row and column means and scales based on the input matrix `X`. It iteratively adjusts these values to minimize the residuals, which represent the difference between the observed and estimated values.
   - `transform(X)`: This method applies the estimated row and column means and scales to the input matrix `X`, resulting in a normalized matrix.

   The result of the `fit_transform` method is assigned to `matrix_train_normalized`, which is the normalized version of the `matrix_train` array. Normalizing the matrix helps to ensure that the data is centered and scaled appropriately, which can improve the performance of collaborative filtering algorithms by making the data more consistent and comparable across different users and items.

In summary, this code snippet initializes a `BiScaler` object with specific parameters and uses it to normalize the training matrix, preparing the data for further analysis or modeling in a collaborative filtering context.

In [9]:
biscaler = BiScaler(scale_rows=False, scale_columns=False, max_iters=50, verbose=False)
matrix_train_normalized = biscaler.fit_transform(matrix_train)

The code below initializes an instance of the `SoftImpute` class with specific parameters. The `SoftImpute` class is designed to perform matrix completion using the SoftImpute algorithm, which is useful for filling in missing values in a matrix, such as a user-item rating matrix in collaborative filtering.

1. **Creating the SoftImpute Object**:
   ```python
   softImpute = SoftImpute(J = 6, maxit = 200, random_seed = 2033, verbose = False)
   ```
   This line creates an instance of the `SoftImpute` class with the following parameters:
   - `J = 6`: This parameter sets the rank of the approximation, meaning the algorithm will use 6 singular values and vectors to approximate the matrix.
   - `maxit = 200`: This parameter sets the maximum number of iterations for the algorithm to 200. The algorithm will stop after 200 iterations if it has not already converged.
   - `random_seed = 2033`: This parameter sets the seed for the random number generator to 2033, ensuring reproducibility of the results. Using a fixed seed allows the same random numbers to be generated each time the code is run.
   - `verbose = False`: This parameter disables verbose output, meaning the algorithm will not print progress messages during execution.

2. **Understanding the SoftImpute Class**:
   The `SoftImpute` class is designed to iteratively estimate missing values in a matrix by performing singular value decomposition (SVD) and applying soft-thresholding to the singular values. The class has several attributes and methods:
   - `__init__`: The constructor method initializes the object with the specified parameters and sets up internal variables.
   - `fit`: This method performs the SoftImpute algorithm on the input matrix `X`, filling in missing values and updating the singular value decomposition components (`U`, `Dsq`, `V`).
   - `suv`: This method computes the product of the left singular vectors and the scaled right singular vectors.
   - `predict`: This method generates the completed matrix by applying the learned SVD components to the input matrix `X`.

By initializing the `SoftImpute` object with these parameters, the code prepares to perform matrix completion on a dataset. The `fit` method will be called later to apply the SoftImpute algorithm to the matrix, filling in missing values and producing a completed matrix that can be used for further analysis or recommendations.

In [10]:
softImpute = SoftImpute(J = 6, maxit = 200, random_seed = 2033, verbose = False)

The code below applies the SoftImpute algorithm to a normalized training matrix to fill in missing values and then transforms the filled matrix back to its original scale. This process is part of preparing the data for collaborative filtering tasks.

1. **Fitting the SoftImpute Model**:
   ```python
   matrix_train_softImpute = softImpute.fit(matrix_train_normalized)
   ```
   This line calls the `fit` method of the `SoftImpute` object on the `matrix_train_normalized` array. The `fit` method performs the SoftImpute algorithm, which iteratively estimates the missing values in the matrix by performing singular value decomposition (SVD) and applying soft-thresholding to the singular values. The method returns the `SoftImpute` object itself, with the learned SVD components (`U`, `Dsq`, `V`) stored as attributes.

2. **Predicting the Filled Matrix**:
   ```python
   matrix_train_filled_normalized = matrix_train_softImpute.predict(matrix_train_normalized, copyto=False)
   ```
   This line calls the `predict` method of the `SoftImpute` object on the `matrix_train_normalized` array. The `predict` method uses the learned SVD components to generate the completed matrix by filling in the missing values. The `copyto=False` parameter indicates that the filled matrix should be returned as a new array rather than modifying the original array in place. The result is assigned to `matrix_train_filled_normalized`, which is the normalized version of the filled training matrix.

3. **Inverse Transforming the Filled Matrix**:
   ```python
   matrix_train_filled = biscaler.inverse_transform(matrix_train_filled_normalized)
   ```
   This line calls the `inverse_transform` method of the `BiScaler` object on the `matrix_train_filled_normalized` array. The `inverse_transform` method reverses the normalization process applied earlier, restoring the matrix to its original scale. This involves undoing any centering and scaling that was applied during the normalization step. The result is assigned to `matrix_train_filled`, which is the filled training matrix in its original scale.

In summary, this code snippet applies the SoftImpute algorithm to fill in missing values in a normalized training matrix and then transforms the filled matrix back to its original scale. This process ensures that the data is complete and ready for further analysis or modeling in a collaborative filtering context. The filled matrix can be used to make recommendations or evaluate the performance of the collaborative filtering model.

In [11]:
matrix_train_softImpute = softImpute.fit(matrix_train_normalized)
matrix_train_filled_normalized = matrix_train_softImpute.predict(matrix_train_normalized, copyto = False)
matrix_train_filled = biscaler.inverse_transform(matrix_train_filled_normalized)

The code below calculates the average value of the training data in a matrix. This is done using the `np.average` function from the NumPy library.

1. **Selecting Training Data**:
   ```python
   matrix_train[train_indices]
   ```
   This part of the code selects the elements of the `matrix_train` array that correspond to the training indices. The `train_indices` were previously determined and represent the positions in the matrix where the training data is located. By indexing `matrix_train` with `train_indices`, the code extracts the relevant subset of the matrix that contains the training data.

2. **Calculating the Average**:
   ```python
   train_average = np.average(matrix_train[train_indices])
   ```
   This line calculates the average of the selected training data using the `np.average` function. The `np.average` function computes the weighted average of the input array. In this case, no weights are provided, so the function calculates the simple arithmetic mean of the selected elements. The result is assigned to the variable `train_average`.

The `np.average` function is versatile and can handle various scenarios, including weighted averages and averaging along specific axes. However, in this context, it is used to compute the overall average of the training data values in the matrix.

By calculating the average value of the training data, the code obtains a summary statistic that can be useful for various purposes, such as normalizing the data, setting baseline predictions, or evaluating the overall rating distribution. This average value provides insight into the central tendency of the training data, which can be important for understanding the dataset and informing subsequent analysis or modeling steps.

In [12]:
train_average = np.average(matrix_train[train_indices])

The code below calculates the mean squared error (MSE) for both the training and validation sets, compares these errors to baseline errors, and prints the R-squared values for both sets. This process evaluates the performance of the collaborative filtering model.

1. **Calculating Validation MSE**:
   ```python
   validation_mse = ((matrix_train_filled[validation_indices] - matrix_incomplete[validation_indices]) ** 2).mean()
   ```
   This line calculates the mean squared error for the validation set. It computes the difference between the predicted ratings (`matrix_train_filled`) and the actual ratings (`matrix_incomplete`) at the validation indices. The differences are squared, and the mean of these squared differences is calculated using the `mean` function. The result is assigned to `validation_mse`.

2. **Calculating Training MSE**:
   ```python
   training_mse = ((matrix_train_filled[train_indices] - matrix_incomplete[train_indices]) ** 2).mean()
   ```
   Similarly, this line calculates the mean squared error for the training set. It computes the difference between the predicted ratings and the actual ratings at the training indices, squares these differences, and calculates the mean. The result is assigned to `training_mse`.

3. **Calculating Baseline Validation MSE**:
   ```python
   validation_mse_baseline = ((train_average - matrix_incomplete[validation_indices]) ** 2).mean()
   ```
   This line calculates the baseline mean squared error for the validation set. It computes the difference between the average training rating (`train_average`) and the actual ratings at the validation indices, squares these differences, and calculates the mean. The result is assigned to `validation_mse_baseline`. This baseline error represents the error of a simple model that always predicts the average training rating.

4. **Calculating Baseline Training MSE**:
   ```python
   training_mse_baseline = ((train_average - matrix_incomplete[train_indices]) ** 2).mean()
   ```
   Similarly, this line calculates the baseline mean squared error for the training set. It computes the difference between the average training rating and the actual ratings at the training indices, squares these differences, and calculates the mean. The result is assigned to `training_mse_baseline`.

5. **Printing R-squared Values**:
   ```python
   print("out-of-sample R2: %.4f, in-sample R2: %.4f." % (1 - validation_mse / validation_mse_baseline, 1 - training_mse / training_mse_baseline))
   ```
   This line prints the R-squared values for both the validation and training sets. The R-squared value is a measure of how well the model's predictions match the actual ratings, with a value of 1 indicating perfect predictions and a value of 0 indicating that the model performs no better than the baseline. The R-squared value is calculated as `1 - (MSE / baseline MSE)`. The `print` function outputs the formatted R-squared values to the console.

In summary, this code snippet evaluates the performance of the collaborative filtering model by calculating the mean squared errors for the training and validation sets, comparing these errors to baseline errors, and printing the R-squared values. This provides a quantitative measure of how well the model predicts the ratings compared to a simple baseline model.

In [13]:
validation_mse = ((matrix_train_filled[validation_indices] - matrix_incomplete[validation_indices]) ** 2).mean()
training_mse = ((matrix_train_filled[train_indices] - matrix_incomplete[train_indices]) ** 2).mean()
validation_mse_baseline = ((train_average - matrix_incomplete[validation_indices]) ** 2).mean()
training_mse_baseline = ((train_average - matrix_incomplete[train_indices]) ** 2).mean()
print("out-of-sample R2: %.4f, in-sample R2: %.4f." % (1 - validation_mse / validation_mse_baseline, 1 - training_mse / training_mse_baseline))

out-of-sample R2: 0.2173, in-sample R2: 0.4981.


The code below prints the value of a specific element in the `matrix_train_filled` array after the matrix completion process. This is done using the `print` function.

1. **Printing a Message with a Specific Matrix Element**:
   ```python
   print("After matrix completion =", matrix_train_filled[526,80])
   ```
   This line uses the `print` function to output a message to the console. The message includes a string and the value of a specific element in the `matrix_train_filled` array. The `matrix_train_filled[526,80]` expression accesses the element at row index 526 and column index 80 of the `matrix_train_filled` array. This element represents the predicted rating for a particular user and item after the matrix completion process.

2. **Understanding the `print` Function**:
   The `print` function is a built-in Python function that outputs the specified values to the console or another output stream. It takes several optional parameters:
   - `*values`: One or more objects to be printed. In this case, the objects are the string `"After matrix completion ="` and the value of `matrix_train_filled[526,80]`.
   - `sep`: A string inserted between the values. The default is a single space (`" "`).
   - `end`: A string appended after the last value. The default is a newline character (`"\n"`).
   - `file`: An object with a `write` method, specifying where to write the output. The default is `sys.stdout`.
   - `flush`: A boolean indicating whether to forcibly flush the stream. The default is `False`.

By calling `print` with the specified arguments, the code outputs a message that includes the value of the matrix element at the specified indices. This is useful for verifying the results of the matrix completion process and ensuring that the predicted ratings are as expected.

In summary, this code snippet prints a message to the console that includes the value of a specific element in the `matrix_train_filled` array. This allows the user to inspect the result of the matrix completion process for a particular user-item pair.

In [14]:
print("After matrix completion =", matrix_train_filled[526,80])

After matrix completion = 0.9967332197811194
